In [ ]:
# -*- coding: utf-8 -*-

"""
=============================================================================
IMPLEMENTACIÓN DEL ALGORITMO RC4 EN PYTHON (SOLO PARA FINES EDUCATIVOS)
=============================================================================

¡¡¡ ADVERTENCIA DE SEGURIDAD !!!

Este algoritmo (RC4) está CRIPTOGRÁFICAMENTE ROTO y es INSEGURO.
Se ha demostrado que tiene múltiples vulnerabilidades graves (como el ataque FMS
que rompió la seguridad WEP) y sesgos en su flujo de salida.

NO LO USE BAJO NINGUNA CIRCUNSTANCIA para proteger datos reales.

Este código se proporciona únicamente con fines académicos y educativos para
entender CÓMO funcionaba el algoritmo y POR QUÉ falló.
Para cifrado simétrico seguro, use AES (con un modo AEAD como GCM)
desde una biblioteca criptográfica auditada (ej. PyCryptodome).
=============================================================================
"""

class RC4:
    """
    Una implementación de la clase RC4 para demostrar su lógica interna.
    """

    def __init__(self, key: bytes):
        """
        Inicializa el cifrador con una llave.
        :param key: La llave secreta, como un objeto 'bytes'.
        """
        # La llave debe tener entre 1 y 256 bytes (aunque RC4 las acepta)
        if not (1 <= len(key) <= 256):
            raise ValueError("La longitud de la llave debe estar entre 1 y 256 bytes.")

        self.key = key
        # Nota: La inicialización del estado (KSA) se hará
        # por separado en cada operación (encrypt/decrypt)
        # para asegurar que el estado es fresco cada vez.
        # Un cifrador de flujo NUNCA debe reutilizar un keystream.

    def _ksa(self) -> list:
        """
        Fase 1: KSA (Key-Scheduling Algorithm) - El "Barajado"
        Configura el estado interno 'S' (la baraja de 256 cartas).
        :return: El estado 'S' inicializado y barajado.
        """
        # 1. Inicializar S como una baraja ordenada (0 a 255)
        S = list(range(256))

        j = 0
        key_len = len(self.key)

        # 2. Barajar la baraja S usando la llave
        for i in range(256):
            # Obtener el byte de la llave correspondiente (dando la vuelta si es necesario)
            key_byte = self.key[i % key_len]

            # Actualizar j
            j = (j + S[i] + key_byte) % 256

            # Intercambiar (swap) S[i] y S[j]
            S[i], S[j] = S[j], S[i]

        return S

    def _prga(self, S: list):
        """
        Fase 2: PRGA (Pseudo-Random Generation Algorithm) - El "Generador"
        Esta es la parte que genera la secuencia cifrante (keystream).
        Usamos un 'generador' de Python (yield) para que produzca un
        flujo infinito de bytes, uno a la vez.

        :param S: El estado 'S' inicializado por el KSA.
        """
        i = 0
        j = 0
        while True:
            # 1. Mover los punteros
            i = (i + 1) % 256
            j = (j + S[i]) % 256

            # 2. Intercambiar S[i] y S[j] (el estado interno sigue mutando)
            S[i], S[j] = S[j], S[i]

            # 3. Calcular el índice del byte de la secuencia cifrante
            t = (S[i] + S[j]) % 256

            # 4. Obtener el byte de la secuencia (keystream)
            k = S[t]

            # 5. Producir (yield) el byte
            yield k

    def crypt(self, data: bytes) -> bytes:
        """
        Proceso de cifrado/descifrado.
        En un cifrado de flujo, el cifrado y el descifrado
        son la misma operación (XOR con el keystream).

        :param data: Los datos a cifrar o descifrar (como 'bytes').
        :return: El resultado cifrado/descifrado (como 'bytes').
        """

        # 1. Inicializar un estado 'S' FRESCO para esta operación.
        #    Esto es vital. Reutilizar 'S' (y por ende el keystream)
        #    es el "pecado capital" de los cifrados de flujo.
        S = self._ksa()

        # 2. Inicializar el generador de la secuencia cifrante
        keystream = self._prga(S)

        # 3. Usar un bytearray para construir el resultado eficientemente
        result = bytearray()

        # 4. Procesar cada byte de los datos
        for byte in data:
            # Pedir el siguiente byte al generador de keystream
            key_byte = next(keystream)

            # Aplicar la magia de XOR
            # (Dato) ⊕ (Keystream) = (Resultado)
            result.append(byte ^ key_byte)

        return bytes(result)

    # Creamos alias para claridad
    encrypt = crypt
    decrypt = crypt

# --- Bloque de Demostración ---
if __name__ == "__main__":

    print("=" * 60)
    print("DEMOSTRACIÓN DEL ALGORITMO RC4 (SOLO FINES EDUCATIVOS)")
    print("¡ADVERTENCIA: RC4 ES INSEGURO Y NO DEBE USARSE JAMÁS!")
    print("=" * 60)

    try:
        # 1. Definir una llave (como bytes)
        # En un escenario real, esta llave NUNCA debe estar en el código.
        llave_secreta = b"EstaEsMiLlave123"

        # 2. Definir un mensaje (como bytes)
        mensaje_claro = b"Este es un mensaje secreto para la clase de criptografia."

        print(f"Mensaje Original: {mensaje_claro.decode('utf-8')}")
        print(f"Llave Secreta:     {llave_secreta.decode('utf-8')}")
        print("-" * 60)

        # 3. Inicializar el cifrador
        cifrador = RC4(llave_secreta)

        # 4. Cifrar el mensaje
        print("Cifrando...")
        mensaje_cifrado = cifrador.encrypt(mensaje_claro)

        # Imprimimos el cifrado en hexadecimal para que sea legible
        print(f"Mensaje Cifrado (hex): {mensaje_cifrado.hex()}")
        print("-" * 60)

        # 5. Descifrar el mensaje
        # ¡IMPORTANTE! Debemos crear una NUEVA instancia del cifrador
        # o llamar a la función de nuevo. Si reutilizáramos el mismo
        # objeto 'cifrador' sin reinicializar el estado, el keystream
        # sería diferente y fallaría.
        # Nuestra implementación reinicializa el KSA en cada llamada
        # a 'crypt()', así que podemos reutilizar el objeto.

        print("Descifrando...")
        descifrador = RC4(llave_secreta) # Por claridad, usamos un objeto nuevo.
        mensaje_descifrado = descifrador.decrypt(mensaje_cifrado)

        print(f"Mensaje Descifrado: {mensaje_descifrado.decode('utf-8')}")
        print("-" * 60)

        # 6. Verificación de éxito
        if mensaje_claro == mensaje_descifrado:
            print("ÉXITO: El mensaje descifrado coincide con el original.")
        else:
            print("FALLO: El mensaje descifrado NO coincide.")

    except Exception as e:
        print(f"Ha ocurrido un error: {e}")

DEMOSTRACIÓN DEL ALGORITMO RC4 (SOLO FINES EDUCATIVOS)
¡ADVERTENCIA: RC4 ES INSEGURO Y NO DEBE USARSE JAMÁS!
Mensaje Original: Este es un mensaje secreto para la clase de criptografia.
Llave Secreta:     EstaEsMiLlave123
------------------------------------------------------------
Cifrando...
Mensaje Cifrado (hex): f7c03de21b34f442b7fdb1b9997a1f4f0263aa1e77126e9301c605febfa881d8c6e9c1e66a6b74d6c6dcd61b14d04a9ddeae43a021817efa9a
------------------------------------------------------------
Descifrando...
Mensaje Descifrado: Este es un mensaje secreto para la clase de criptografia.
------------------------------------------------------------
ÉXITO: El mensaje descifrado coincide con el original.
